# Fine Tuning: ViT para clasificar flores

## Instalación de librerias

In [ ]:
!pip install -q datasets transformers transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoImageProcessor
from torchvision.transforms import (CenterCrop, Compose, Normalize, RandomHorizontalFlip,
                                    RandomResizedCrop, Resize, ToTensor,)
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
import torch
import numpy as np

## Inicio de sesión: Hugginface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Carga de Datos + Preprocesamiento

In [ ]:
dataset = load_dataset("DeadPixels/DPhi_Sprint_25_Flowers")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2589 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/864 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/864 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 2589
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 864
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 864
    })
})

In [ ]:
dataset["train"].features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['daisy', 'dandelion', 'rose', 'sunflower', 'tulip'], id=None)}

Se crean dos diccionarios que van a servir como traductores entre ids y etiquetas

In [ ]:
labels = dataset["train"].features["label"].names
label_to_id, id_to_label = dict(), dict()
for i, label in enumerate(labels):
    label_to_id[label] = i
    id_to_label[i] = label

In [ ]:
metric = load_metric("accuracy")

<ipython-input-8-21f3ecd7ee32>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


El modelo a usar es **ViT (Vision Transformer)**. Es un modelo para clasificar imágenes que utiliza una arquitectura similar a los Transformers

In [ ]:
checkpoint = "google/vit-base-patch16-224-in21k"
batch_size = 16

In [ ]:
image_processor  = AutoImageProcessor.from_pretrained(checkpoint)
image_processor

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

Se realizan transformaciones a las imágenes para hacer más robusto al modelo. Dentro de las transformaciones tenemos: cambio de tamaño, volteado o cortado.

Ciertas se aplican al train y ciertas al validation

In [ ]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

### Separacion: train & test

In [ ]:
splits = dataset["train"].train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

## Entrenamiento de modelo

In [ ]:
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    label2id=label_to_id,
    id2label=id_to_label,
)

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
model_name = checkpoint.split("/")[-1]

args = TrainingArguments(
    f"finetuned-vit-flowers",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [ ]:
train_results = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.123600,0.150866,0.972973
2,0.107700,0.136525,0.965251


In [ ]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

events.out.tfevents.1701919375.6e779dc37d45.320.3:   0%|          | 0.00/7.26k [00:00<?, ?B/s]

***** train metrics *****
  epoch                    =        2.96
  total_flos               = 497986660GF
  train_loss               =      0.1215
  train_runtime            =  0:05:16.42
  train_samples_per_second =       22.09
  train_steps_per_second   =       0.341


## Carga del modelo

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/manoh2f2/finetuned-vit-flowers/tree/main/'